<a href="https://colab.research.google.com/github/dettystanly7676/Python_Practice_Repo/blob/main/Case_Study_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Load SpaceX Launch Data from API

In [35]:
import pandas as pd
import numpy as np
import sqlite3
import requests
import random

Use https://api.spacexdata.com/v4/launches

In [27]:
SpaceX_data = pd.read_json('https://api.spacexdata.com/v4/launches')
SpaceX_data.head()

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,...,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,...,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,...,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b
2,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,...,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight'...",True,False,None,5eb87cdbffd86e000604b32c
3,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,1.0,[],Ratsat was carried to orbit on the first succe...,...,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",True,False,None,5eb87cdbffd86e000604b32d
4,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,1.0,[],None,...,2009-07-13T03:35:00.000Z,1247456100,2009-07-13T15:35:00+12:00,hour,False,"[{'core': '5e9e289ef359184f103b2627', 'flight'...",True,False,None,5eb87cdcffd86e000604b32e


 Extract relevant columns: name, date_utc, success, details, rocket

In [28]:
SpaceX_data_Extracted = SpaceX_data[['name', 'date_utc', 'success', 'details', 'rocket']]
SpaceX_data_Extracted.head()

,name,date_utc,success,details,rocket
0,FalconSat,2006-03-24T22:30:00.000Z,0.0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb
1,DemoSat,2007-03-21T01:10:00.000Z,0.0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb
2,Trailblazer,2008-08-03T03:34:00.000Z,0.0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb
3,RatSat,2008-09-28T23:15:00.000Z,1.0,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb
4,RazakSat,2009-07-13T03:35:00.000Z,1.0,None,5e9d0d95eda69955f709d1eb


 Convert date_utc to datetime and extract the year


In [32]:
date_time_format = pd.to_datetime(SpaceX_data['date_utc'])
print(date_time_format.head())

Extracted_year = date_time_format.dt.year
print(Extracted_year.head())

0   2006-03-24 22:30:00+00:00
1   2007-03-21 01:10:00+00:00
2   2008-08-03 03:34:00+00:00
3   2008-09-28 23:15:00+00:00
4   2009-07-13 03:35:00+00:00
Name: date_utc, dtype: datetime64[ns, UTC]
0    2006
1    2007
2    2008
3    2008
4    2009
Name: date_utc, dtype: int32


# Step 2: Load Rocket Metadata

In [15]:
rocket_data = pd.read_json(' https://api.spacexdata.com/v4/rockets')
rocket_data

,height,diameter,mass,first_stage,second_stage,engines,landing_legs,payload_weights,flickr_images,name,...,stages,boosters,cost_per_launch,success_rate_pct,first_flight,country,company,wikipedia,description,id
0,"{'meters': 22.25, 'feet': 73}","{'meters': 1.6800000000000002, 'feet': 5.5}","{'kg': 30146, 'lb': 66460}","{'thrust_sea_level': {'kN': 420, 'lbf': 94000}...","{'thrust': {'kN': 31, 'lbf': 7000}, 'payloads'...","{'isp': {'sea_level': 267, 'vacuum': 304}, 'th...","{'number': 0, 'material': None}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...","[https://imgur.com/DaCfMsj.jpg, https://imgur....",Falcon 1,...,2,0,6700000,40,2006-03-24,Republic of the Marshall Islands,SpaceX,https://en.wikipedia.org/wiki/Falcon_1,The Falcon 1 was an expendable launch system p...,5e9d0d95eda69955f709d1eb
1,"{'meters': 70, 'feet': 229.6}","{'meters': 3.7, 'feet': 12}","{'kg': 549054, 'lb': 1207920}","{'thrust_sea_level': {'kN': 7607, 'lbf': 17100...","{'thrust': {'kN': 934, 'lbf': 210000}, 'payloa...","{'isp': {'sea_level': 288, 'vacuum': 312}, 'th...","{'number': 4, 'material': 'carbon fiber'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://farm1.staticflickr.com/929/2878733830...,Falcon 9,...,2,0,50000000,98,2010-06-04,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_9,Falcon 9 is a two-stage rocket designed and ma...,5e9d0d95eda69973a809d1ec
2,"{'meters': 70, 'feet': 229.6}","{'meters': 12.2, 'feet': 39.9}","{'kg': 1420788, 'lb': 3125735}","{'thrust_sea_level': {'kN': 22819, 'lbf': 5130...","{'thrust': {'kN': 934, 'lbf': 210000}, 'payloa...","{'isp': {'sea_level': 288, 'vacuum': 312}, 'th...","{'number': 12, 'material': 'carbon fiber'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://farm5.staticflickr.com/4599/385838292...,Falcon Heavy,...,2,2,90000000,100,2018-02-06,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_Heavy,With the ability to lift into orbit over 54 me...,5e9d0d95eda69974db09d1ed
3,"{'meters': 118, 'feet': 387}","{'meters': 9, 'feet': 30}","{'kg': 1335000, 'lb': 2943000}","{'thrust_sea_level': {'kN': 128000, 'lbf': 287...","{'thrust': {'kN': 1957, 'lbf': 440000}, 'paylo...","{'isp': {'sea_level': 330, 'vacuum': 380}, 'th...","{'number': 6, 'material': 'stainless steel'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://live.staticflickr.com/65535/489541389...,Starship,...,2,0,7000000,0,2021-12-01,United States,SpaceX,https://en.wikipedia.org/wiki/SpaceX_Starship,Starship and Super Heavy Rocket represent a fu...,5e9d0d96eda699382d09d1ee


Extract id, name, type, active, and stages

In [19]:
Rocket_data_Extracted = rocket_data[['id', 'name', 'active', 'stages']]
Rocket_data_Extracted

,id,name,active,stages
0,5e9d0d95eda69955f709d1eb,Falcon 1,False,2
1,5e9d0d95eda69973a809d1ec,Falcon 9,True,2
2,5e9d0d95eda69974db09d1ed,Falcon Heavy,True,2
3,5e9d0d96eda699382d09d1ee,Starship,False,2


# Step 3: Merge Launch and Rocket Data


In [55]:
# Merged_Data_Frame = pd.merge(SpaceX_data, rocket_data, on='id', how='inner')
# Merged_Data_Frame.head()
Merged_Data_Frame = pd.merge(
    SpaceX_data_Extracted,            # Left DataFrame (Launches)
    Rocket_data_Extracted,            # Right DataFrame (Rockets)
    left_on='rocket',       # Use the 'rocket' column from SpaceX_data
    right_on='id',   # Use the 'rocket_id' column from rocket_data (assuming you renamed the original 'id')
    how='left',             # Use a LEFT merge to keep all launch records (prevents empty results from inner join)
    suffixes=('_launch', '_rocket')
)
Merged_Data_Frame.head()

,name_launch,date_utc,success,details,rocket,id,name_rocket,active,stages
0,FalconSat,2006-03-24T22:30:00.000Z,0.0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,Falcon 1,False,2
1,DemoSat,2007-03-21T01:10:00.000Z,0.0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,Falcon 1,False,2
2,Trailblazer,2008-08-03T03:34:00.000Z,0.0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,Falcon 1,False,2
3,RatSat,2008-09-28T23:15:00.000Z,1.0,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,Falcon 1,False,2
4,RazakSat,2009-07-13T03:35:00.000Z,1.0,None,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,Falcon 1,False,2


# Step 4: Add Simulated Country Information
 Add a new column country and randomly assign one of these values:

['USA', 'Russia', 'India', 'China', 'France']


In [56]:
COUNTRIES = ['USA', 'Russia', 'India', 'China', 'France']
Merged_Data_Frame['country'] = [random.choice(COUNTRIES) for _ in range(len(Merged_Data_Frame))]
Merged_Data_Frame.head()

,name_launch,date_utc,success,details,rocket,id,name_rocket,active,stages,country
0,FalconSat,2006-03-24T22:30:00.000Z,0.0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,Falcon 1,False,2,India
1,DemoSat,2007-03-21T01:10:00.000Z,0.0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,Falcon 1,False,2,India
2,Trailblazer,2008-08-03T03:34:00.000Z,0.0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,Falcon 1,False,2,India
3,RatSat,2008-09-28T23:15:00.000Z,1.0,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,Falcon 1,False,2,China
4,RazakSat,2009-07-13T03:35:00.000Z,1.0,None,5e9d0d95eda69955f709d1eb,5e9d0d95eda69955f709d1eb,Falcon 1,False,2,India


# Step 5: Store Merged Data in SQLite3

Use sqlite3 to create a connection and save the merged DataFrame as a table

named launches

In [58]:
DB_Name = 'Data_Analytics.db'
Table_Name = 'Merged_Data_Frame'
conn = sqlite3.connect(DB_Name)
Merged_Data_Frame.to_sql(
    Table_Name,
    conn,
    if_exists='replace',
    index=False
)


205

In [61]:
cursor = conn.cursor()
cursor.execute('select * from Merged_Data_Frame')
# rows = cursor.fetchall()
# for row in rows:
#   print(row)

# Step 6: Run SQL Queries on the Data to analyze
1. Launches by Country


In [64]:
cursor.execute('select country from Merged_Data_Frame group by country')
rows = cursor.fetchall()
for row in rows:
  print(row)

('China',)
('France',)
('India',)
('Russia',)
('USA',)


2. Which year had the highest number of launches

In [68]:
date_time_format = pd.to_datetime(SpaceX_data['date_utc'])
Extracted_year = date_time_format.dt.year

cursor.execute('select SUBSTR(date_utc, 1, 4) AS launch_year from Merged_Data_Frame group by launch_year')
